In [2]:
import os

In [3]:
pwd

'c:\\I39106\\DSP\\Text-Summarization_NLP-main\\research'

In [4]:
os.chdir("../")

In [5]:
pwd

'c:\\I39106\\DSP\\Text-Summarization_NLP-main'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [8]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(self,  config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH):
            
            self.config = read_yaml(config_filepath)
            self.params= read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_intgestion_config= DataIngestionConfig(
                root_dir = config.root_dir,
                source_URL= config.source_URL,
                local_data_file=config.local_data_file,
                unzip_dir = config.unzip_dir
        )
        return data_intgestion_config

In [10]:
## Downloading data 

import os
import urllib.request as request
import zipfile
from TextSummarizer.logging import logger
from TextSummarizer.utils.common import get_size


In [12]:

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config= config



    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f" File already exist of size: {get_size(Path(self.config.local_data_file))}")


    ## Unzip file

    def extract_zip_file(self):
        unzip_path =  self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok= True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-06-06 15:23:57,015: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-06 15:23:57,022: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-06 15:23:57,031: INFO: common: created directory at: artifacts]
[2025-06-06 15:23:57,031: INFO: common: created directory at: artifacts/data_ingestion]
[2025-06-06 15:23:59,785: INFO: 1324717884: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 8266778
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "39ea5838f69b124e37ccceb0e1fe5bc61ead080c65244d7cb8acffdc9f5dc521"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 27D6:B467F:392B78:70D5B2:6842BAB6
Accept-Ranges: bytes
Date: Fri, 06 Jun 2025 09:53:59 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10234-MAA
X-Cache: MI